# Make a neural network for reading handwritten digits

Here a neural network is constructed and trained on MNIST data.

It is a very simple neural network: Two hidden layers, fully connected.

MNIST contains images of 28x28 pixels and a corresponding label of which number it is (0-9).

In [37]:
import torch
import torch.nn as nn

## Get the MNIST data

In [ ]:
import torchvision
# NOTE: ToTensor scales to [0,1] interval
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
mnist_full_train=torchvision.datasets.FashionMNIST(root='MNIST_dir', train=True, transform=transform, download=True)

image_dim=mnist_full_train[0][0].shape[1]
print('Number of pixels in images:',image_dim,'x',image_dim)


In [13]:
# Select the first 1000 images for training
train_set = torch.utils.data.Subset(mnist_full_train, list(range(1000)))

# Make a "data loader" that will return batches of images and labels
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)


## Make the neural network

We use the nn.Sequential, which puts the modules one after the other.

Because the images are 2D tensors, the first module flattens the images to 1D - from 28x28 to an array of 784 numbers between 0 and 1 (gray scale).

In [39]:
my_network = nn.Sequential(
    # First, flatten image
    nn.Flatten(),
    # One linear layer with 784 inputs and 100 units
    nn.Linear(image_dim*image_dim,100),
    # ReLU activation function
    nn.ReLU(),
    # Next layer must now have 100 inputs and we chose 50 units in this layer
    nn.Linear(100,50),
    nn.ReLU(),
    # Final layer must have 10 output units, one for each digit 
    nn.Linear(50,10)
    )

## Set up the loss function and optimizer

In [ ]:
# The cross entropy loss function combines softmax and loss into one
loss_function = nn.CrossEntropyLoss()

# The optimizer must know which parameters to optimize
# The trainable parameters of a network is returned by the parameters() method
optimizer = torch.optim.Adam(my_network.parameters())


## Train the model

In [ ]:

# Here we run 50 epochs
for epoch in range(50):
    # Each time we call the train_loader it returns one batch of examples
    for image,label in train_loader:
        ### Standard training sequence that can be used always ################
        # Reset all the gradients to zero
        optimizer.zero_grad()
        # Get the outputs of the neural network for the batch
        y = my_network(image)
        # Calculate the loss
        loss = loss_function(y,label)
        # Do the back-propagation
        loss.backward()
        # Update the weights
        optimizer.step()
        ### End of standard sequence ##########################################

    if (epoch+1)%10==0:
        print('Epoch:',epoch,'Loss',loss.item())




## Check if results make sense

In [ ]:
# Pick an arbitrary example from the training set
image,label = train_set[7]
y = nn.functional.softmax(my_network(image).squeeze(),dim=0)
print('Actual label',label)
print('Output after softmax:')
for i in range(10):
    print(i,': ','%.5f' % y[i].item())